## Package import and define parameters

Below, you will define the basic parameters for your database and subcellular structures of interest. You will also initialize the tables in your database to hold the data for your objects of interest.

In general, the sections of this notebook can be run separately from each other. However, you'll want to update and run the parameter defining cell before jumping ahead to other sections. If you get an error stating that something has not been defined, it means you probably need to return to the parameters cell and run it. 


In [ ]:
## Define database and structural information

# database info
database_name = 'demo'

# path to find data

FILE_PATH = '/data/'

# Subcellular structures of interest. We recommend keeping all names in letters only 
# and avoiding upper-case letters. You should keep your smFISH images named 'rna'
# These names should match the sub-folders in your data folder
structures = ['rna', 'centrosomes']


# Each structure folder should have two sub-folders, 'raw-data' and 'segmentations'
# You can use different names for these folders if you like -- but update the variables below 
# if you do

raw_data_dir = 'raw-data'
segmentation_dir = 'segmentations'

# If your segmentation files have a different suffix than your raw data files, 
# update the variable below with the suffix
# The suffix below is the suffix that is generated by the Allen Cell Segmenter batch segmentation process
# If your segmentation files have the same suffix as your raw data filenames, then change this variable 
# to an empty string, e.g. segmentation_file_suffix = ''

segmentation_file_suffix = '_struct_segmentation.tiff'


# Update the scale parameters below for the number of microns per pixel in the xy plane and the z plane
xy_scale = 0.065 # microns per pixel in the xy dimension
z_scale = 0.25   # microns between each z step



In [ ]:
# Create a database for your experiment
# Only needs to be run once - will throw an error if run multiple times, since the database won't be overwritten

import os
import psycopg2
from psycopg2 import sql

conn = psycopg2.connect('postgresql://'+os.environ['POSTGRES_USER']+':'+os.environ['POSTGRES_PASSWORD']+'@'+"db"+':'+'5432'+'/'+os.environ['POSTGRES_DB'])
conn.autocommit = True

cursor = conn.cursor()

cursor.execute(sql.SQL("CREATE DATABASE {database}").format(database=sql.Identifier(database_name)))

cursor.close()
conn.close()


## Object data extraction

These cells extracts basic data about the subcellular structure objects and insert it into the postgres database.

**Note that images are not reprocessed.**

If you need to delete object data for a structure in a particular image, use the `delete_data_db(image_name, structure, database_name)` function

In a new cell, you would run:
```bash
from pipeline import delete_data_db
image_name = ''
structure = ''
delete_data_db(image_name, structure, database_name)
```
Update the image_name and structure variables with the respective image names and structures.

In [ ]:
# Python package
import os
import psycopg2

# Functions from the pipeline.py module
from pipeline import create_postgres_table, test_data_db, extract_object_properties, insert_object_data


In [ ]:
# This cell creates postgres tables to hold data for the structures of interest
# Table names are automatically assigned based on your structures of interest

for structure in structures:
    create_postgres_table(structure, database_name)


In [ ]:
# This cell navigates through the files in your data directories and extracts basic object data such as 
# area and integrated intensity for each structure 

for structure in structures:
    print(structure)
    
    segmentations_path = os.path.join(FILE_PATH, structure, segmentation_dir)

    for seg_img_name in os.listdir(segmentations_path):
        if not seg_img_name[0] == '.':

            ins_img_name = seg_img_name

            if segmentation_file_suffix:
                ins_img_name = seg_img_name[0:-(len(segmentation_file_suffix))] + '.tif'

            # check if the image has been processed

            
            if test_data_db(ins_img_name, structure, database_name):
                print('{structure} data for {image_name} has already been processed and will not be re-processed'.format(structure=structure, image_name=ins_img_name))

            else:
                seg_img_path = os.path.join(segmentations_path, seg_img_name)
                ins_img_path = os.path.join(FILE_PATH, structure, raw_data_dir, ins_img_name)

                # extract the object properties for that image
                object_data_list = extract_object_properties(seg_img_path, ins_img_path, ins_img_name, xy_scale, z_scale)

                # save data to database
                insert_object_data(structure, object_data_list, database_name)

## Measure distances between two subcellular structures

Below, you will define two subcellular structures. Structure 1 is the "object of interest." For a given image, each structure 1 object will be measured for how far it is to each structure 2 object in the same image. The closest structure 2 object will be identified and the distance to that object will be recorded in the database. 

You will define which objects you are interested in distance measurements for. Do you want to know how far away the RNA is from the closest centrosome, nucleus, etc? If you want to do measure distances from RNA to more than one subcellular object, then you would repeat this process in a separate cell.


In [ ]:
# package import for distance measurements


# local packages
from pipeline import measure_distance_by_obj, add_distance_columns


In [ ]:
## Define the parameters 

# define which structures to measure between
# Format is: (structure_1, structure_2), where each structure_1 object will be measured for
# the shortest distance to structure_2
# You can include multiple pairs of structures to measure between 
# e.g. [('rna', 'centrosomes'), ('rna', 'nuclei')] would measure the distance
# for each rna object to the nearest centrosome and the nearest nucleus

structure_measurement_tuples = [('rna', 'centrosomes')]


# variable to determine if parallel processing is used
parallel_processing_bool = True

# variable to determine number of objects to measure using surface-to-surface measurements
# increase this number if both subcellular structures that are measured are densely packed
# if at least one object is sparsely distributed, you can decrease this number
number_centroid_measure = 3

In [ ]:

for structure_tuple in structure_measurement_tuples:
    structure_1 = structure_tuple[0]
    structure_2 = structure_tuple[1]

    print('Measuring distances between ' + structure_1 + ' and ' + structure_2)

    add_distance_columns(structure_1, structure_2, database_name)

    measure_distance_by_obj(structure_1, structure_2, parallel_processing_bool, number_centroid_measure, database_name)


## Create an images table with metadata for each image

Your raw-data images folder likely contains images of a control RNA or control biological condition together with images from your experimental condition. You may also have multiple biological replicates. You can use postgres to help track where each image comes from -- e.g. is it an image of control RNA or an experimental RNA?

We recommend storing these data in a .csv file. You can create this file using Excel and then "Save As" .csv (you specifically want the "Comma Separated Values" file type and not the "CSV UTF-8" file type). You can then load the .csv file into postgres using the code below:





In [ ]:
# import packages

import csv
import os.path

import psycopg2
from sqlalchemy import create_engine

import pandas as pd


In [ ]:
# define parameters

# path to the directory containing your csv file
FILE_PATH = '/data/'

# name of your csv file
csv_name = 'raw-data-metadata.csv'



In [ ]:
# import data into pandas dataframe from csv 
csv_filepath = os.path.join(FILE_PATH, csv_name)

image_df = pd.read_csv(csv_filepath)

image_df.head()

In [ ]:
# create postgres connection
engine = create_engine('postgresql://'+os.environ['POSTGRES_USER']+':'+os.environ['POSTGRES_PASSWORD']+'@'+"db"+':'+'5432'+'/'+database_name)


# save the data to the postgres database
# this will overwrite an existing table with this 
image_df.to_sql('images', con=engine, if_exists = 'replace')

## Normalize single molecule fluorescence data - optional

Here we provide a generalizable tool to estimate the number of molecules per object for single molecule fluorescence data.

We apply this tool in this example to normalize single molecule RNA FISH data. We recommend that you have a column in your images table that details the type of single molecule data, which allows you to normalize the single molecule separately for different biological species. If you only have one biological condition, you should still include this table, but fill in only one biological type. 

The key to this technique is to have a way to identify single molecules from your collection of objects. We use the volume of the object, and have empirically determined that single molecules of our smFISH data typically contain between 20 and 100 pixels. This threshold will be based on your imaging parameters and your optical system.


In [ ]:
# import packages
import psycopg2
from psycopg2 import sql
import os

In [ ]:
# define parameters

# name of the table that contains your single molecule data. 
single_molecule_table = 'rna'

# the average integrated intensity is calculated for each rna_type
single_molecule_type_column = 'rna_type'

# these thresholds identify single molecules using the area feature
# of each object
# You will need to define these thresholds empirically, based on your data
lower_threshold = 20
upper_threshold = 100


In [ ]:
# first add a column for the normalized intensity data to your structure table of interest


add_normalized_col = sql.SQL("""ALTER TABLE {single_molecule_table} 
                                ADD COLUMN IF NOT EXISTS normalized_intensity REAL;""").format(
                                    single_molecule_table = sql.Identifier(single_molecule_table))

conn = psycopg2.connect('postgresql://'+os.environ['POSTGRES_USER']+':'+os.environ['POSTGRES_PASSWORD']+'@'+"db"+':'+'5432'+'/'+database_name)
cur = conn.cursor()

cur.execute(add_normalized_col)
conn.commit()
cur.close()
conn.close()

In [ ]:
# get the single molecule types types from the database

single_molecule_type_query = sql.SQL("""SELECT DISTINCT ({single_molecule_type_column}) 
                            FROM images;""").format(
                                single_molecule_type_column=sql.Identifier(single_molecule_type_column))

conn = psycopg2.connect('postgresql://'+os.environ['POSTGRES_USER']+':'+os.environ['POSTGRES_PASSWORD']+'@'+"db"+':'+'5432'+'/'+database_name)
cur = conn.cursor()

cur.execute(single_molecule_type_query)
single_molecule_types =  [single_molecule_type[0] for single_molecule_type in cur.fetchall()]

cur.close()
conn.close

print('The single molecule types in your database are:\n' + str(single_molecule_types))

In [ ]:
# this query calculates the average total intensity of objects between the upper and lower thresholds set in the 
# parameters column. It then divides every value in the total intensity column by this average and sets the 
# normalized intensity column to this result


for single_molecule_type in single_molecule_types:
    normalization_sql = sql.SQL("""UPDATE {single_molecule_table} 
    SET normalized_intensity = total_intensity / 
    (SELECT avg({single_molecule_table}.total_intensity) FROM {single_molecule_table}
                                                    INNER JOIN images 
                                                    ON {single_molecule_table}.name = images.name 
                                                    WHERE images.{single_molecule_type_column} = %(single_molecule_type)s
                                                    AND {single_molecule_table}.area >= %(lower_threshold)s
                                                    AND {single_molecule_table}.area < %(upper_threshold)s) 
    FROM images 
    WHERE {single_molecule_table}.name = images.name 
    AND images.{single_molecule_type_column} = %(single_molecule_type)s 
    AND {single_molecule_table}.area >= %(lower_threshold)s;""").format(
                    single_molecule_table=sql.Identifier(single_molecule_table),
                    single_molecule_type_column=sql.Identifier(single_molecule_type_column))
    
    conn = psycopg2.connect('postgresql://'+os.environ['POSTGRES_USER']+':'+os.environ['POSTGRES_PASSWORD']+'@'+"db"+':'+'5432'+'/'+database_name)

    cur = conn.cursor()
    
    cur.execute(normalization_sql, {'single_molecule_type' : single_molecule_type, 'lower_threshold': lower_threshold, 'upper_threshold': upper_threshold})
    conn.commit()
    cur.close()
    conn.close()


## DATA PROCESSING :

In this section, we provide tools to calculate the % of RNA relative to distance and the cumulative % of RNA relative to distance.

The first step is to set parameters, which are required for both calculations. If you have single molecule data and want to calculate the % of structure_1 in objects that contain at least *x* number of molecules, then you would set `granule_bool = True` and `granule_threshold = x`

In [ ]:
# package import
import os


In [ ]:
# define parameters

# update the strings that describe your structures of interest
# for every structure 1 object, the closest distance to a structure 2 object is measured

structure_1 = 'rna'
structure_2 = 'centrosomes'

# update the column name in your images table that holds data for your image names
image_name_column = 'name'

# the step size between % structure_1 measurements; in microns
step_size = 0.05

# optional: distance threshold
distance_threshold = 5

# Optional: if desired, pipeline will calculate % of structure 1 objects that contain > a user-defined 
# threshold at each increment. If you set granule_bool = True, you should have single molecule normalized data
# in your structure_1 table

granule_bool = True
granule_threshold = 4

# parameters for saving data. Default is to save in the directory containing your raw-data
# and segmentations in a folder named data within a folder named output 

csv_output_dir = os.path.join(FILE_PATH, 'output', 'data')

# make an output directory if it doesn't already exist

if not os.path.isdir(csv_output_dir):
    os.makedirs(csv_output_dir)


print("The directory where distribution data will be saved is:\n" + csv_output_dir)


### Calculate % of structure 1 relative to distance from structure 2

In this section, we calculate the distribution of structure 1 relative to the distance from a subcellular structure of interest. 

We work with fractions of structure 1 relative to the total fluorescence for structure 1. This approach normalizes for differences in intensity. 

An optional threshold can be provided for the upper distance threshold. 

For users with normalized single molecule data, the % of structure 1 in objects > a user-defined threshold relative to distance can also be calculated. Set granule_bool = True and granule_threshold = desired_threshold (e.g. 4) to include this calculation

In [ ]:
from pipeline import calculate_fraction_rna, save_csv


In [ ]:
structure_1_distribution_df = calculate_fraction_rna(structure_1, structure_2, image_name_column, distance_threshold, granule_bool, granule_threshold, database_name)

structure_1_distribution_df.head()

In [ ]:
# here we save this csv for later plotting and visualization
# note that the file will not be overwritten

save_csv('fraction_' + structure_1 + '_per_distance.csv', csv_output_dir, structure_1_distribution_df)

### Calculate cumulative % of structure 1 relative to distance from structure 2

In this section, we calculate the cumulative distribution of structure 1 relative to the distance from a subcellular structure of interest. 

This workflow will calculate the % of total fluorescence for structure 1 at 0 microns and then at regular intervals defined by the "step_size" parameter up to the distance threshold, if you choose to use a distance threshold. For example, if distance_threshold = 5 and the step_size = 0.05, then this code will calculate the % RNA and % RNA in granules at 0, 0.05, 0.10, 0.15 microns, etc., up to 5 microns for each image.

If you choose not to define an upper distance threshold, then the % of RNA is calculated at intervals defined by the step size from 0 microns up to the maximum distance_from_structure_2 for each image.


In [ ]:
# local packages
from pipeline import calculate_distributions_by_image, save_csv


In [ ]:
# calculate the cumulative distributions and store in a dataframe 
# this calculation can take about 15 minutes per image (depending on the density of data for structure 1)

structure_1_distribution_df = calculate_distributions_by_image(distance_threshold, granule_bool, granule_threshold, step_size, image_name_column, structure_1, structure_2, database_name)

structure_1_distribution_df.head()

In [ ]:
# here we save the cumulative percentage for later plotting and visualization
# files will not be overwritten

save_csv('cumulative_structure_1_per_distance.csv', csv_output_dir, structure_1_distribution_df)


## VISUALIZATION :

This section provides code to visualize your data. Plots can be saved for publication. 

In [ ]:
# package imports

import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
import os


In [ ]:
# parameters

# define the names of your cumulative distribution data and fractional distribution data
fractional_distribution_data_filename = 'fraction_structure_1_per_distance.csv'
cumulative_distribution_data_filename = 'cumulative_structure_1_per_distance.csv'

# define your output directories
data_output_dir = os.path.join(FILE_PATH, 'output', 'data')
plots_output_dir =  os.path.join(FILE_PATH, 'output', 'plots')

# make a plots output directory if it doesn't already exist
if not os.path.isdir(plots_output_dir):
    os.makedirs(plots_output_dir)



In [ ]:
# Here we define a function to save your plots. We choose to save at 600 dpi as .pdf files, which work well with
# Adobe Illustrator files for figure creation. You can change this function so as desired so that your plots will
# be saved consistently

def save_plot(plot_fn, plots_output_dir, plot):
    """ This function takes two strings as inputs and a matplotlib plot object. plot_fn describes the desired filename
    plots_output_dir is the directory to save the plot. plot is a variable containing your plot
    
    This function will not overwrite data
    
    The function tests if a file exists in the plots_output_dir. If a file exists, it prints a message and does nothing
    If a file does not a exist, the plot is saved
    
    Returns nothing
    """
    
    if os.path.isfile(plots_output_dir + '/' + plot_fn):
        print('Plot already saved and will not be saved again')
    else:
        plot.savefig(plots_output_dir + '/' + plot_fn, bbox_inches = 'tight', dpi = 600, format = 'pdf', transparent = True)
        
    return None


In [ ]:
# load data from your csv file. This appraoch allows you to avoid re-calculating the distributions each time you 
# want to plot your data, which is time consuming
# We'll start with the fractional distribution data 

fractional_distribution_df = pd.read_csv(data_output_dir + '/' + fractional_distribution_data_filename)
fractional_distribution_df.head(10)

In [ ]:
# Here we plot the % RNA distribution at each distance as mean (dark line) +/- sd (shading) using the Seaborn library and matplotlib
# https://seaborn.pydata.org/examples/index.html
# There are a lot of options in Seaborn. In this example, we separate the plots into columns based on the variable 
# "cycle" and adjust the color using the rna_type, in order to compare our experimental RNA (cen) to control (gapdh)


sns.set_style("ticks")
sns.set_context("paper")
fractions_structure_1_plt = sns.relplot(x = 'distance', 
                                        y = 'percent_distance', 
                                        hue = 'rna_type', 
                                        col = 'cycle', 
                                        col_order = ['interphase', 'metaphase'], 
                                        ci = "sd", 
                                        kind="line", 
                                        data = fractional_distribution_df);

plt.xlim(0)
plt.ylim(0)


In [ ]:
# Now use your function to save the plot

save_plot('fractions_structure_1.pdf', plots_output_dir, fractions_structure_1_plt)

In [ ]:
# Here we adjust the plot's x-axis range to 0 - 0.5 micrometers, to emphasize the enrichment of cen near centrosomes
# this is achieved using the plt.xlim(0,0.5) code

sns.set_style("ticks")
sns.set_context("paper")
fractions_structure_1_plt = sns.relplot(x = 'distance', 
                                        y = 'percent_distance', 
                                        hue = 'rna_type', 
                                        col = 'cycle', 
                                        col_order = ['interphase', 'metaphase'], 
                                        ci = "sd", 
                                        kind="line", 
                                        data = fractional_distribution_df);

plt.xlim(0, 0.5)
plt.ylim(0)


In [ ]:
# save the plot 

save_plot('fractions_structure_1_less-than-0.5.pdf', plots_output_dir, fractions_structure_1_plt)

In [ ]:
# OPTIONAL -- for those with normalized single molecule data, you can plot the distribution of granule data

sns.set_style("ticks")
sns.set_context("paper")
fractions_granule_structure_1_plt = sns.relplot(x = 'distance', 
                                        y = 'percent_granule', 
                                        hue = 'rna_type', 
                                        col = 'cycle', 
                                        col_order = ['interphase', 'metaphase'], 
                                        ci = "sd", 
                                        kind="line", 
                                        data = fractional_distribution_df);

plt.xlim(0)
plt.ylim(0)



In [ ]:
# save the plot 

save_plot('fractions_granule_structure_1.pdf', plots_output_dir, fractions_granule_structure_1_plt)

In [ ]:
# can reset the x scale to better visualize

sns.set_style("ticks")
sns.set_context("paper")
fractions_granule_structure_1_plt = sns.relplot(x = 'distance', 
                                        y = 'percent_granule', 
                                        hue = 'rna_type', 
                                        col = 'cycle', 
                                        col_order = ['interphase', 'metaphase'], 
                                        ci = "sd", 
                                        kind="line", 
                                        data = fractional_distribution_df);

plt.xlim(0, 0.5)
plt.ylim(0)



In [ ]:
# save the plot 

save_plot('fractions_granule_structure_1-less-than-0.5.pdf', plots_output_dir, fractions_granule_structure_1_plt)

In [ ]:
# We can repeat the same plotting process for the cumulative distribution data
# We adjust the name of the csv file for data import

cumulative_distribution_df = pd.read_csv(data_output_dir + '/' + cumulative_distribution_data_filename)
cumulative_distribution_df.head(10)



In [ ]:
# plot the cumulative distribution 

sns.set_style("ticks")
sns.set_context("paper")
fractions_structure_1_plt = sns.relplot(x = 'distance', 
                                        y = 'percent_total_structure_1', 
                                        hue = 'rna_type', 
                                        col = 'cycle', 
                                        col_order = ['interphase', 'metaphase'], 
                                        ci = "sd", 
                                        kind="line", 
                                        data = cumulative_distribution_df);

plt.xlim(0)
plt.ylim(0,100)


In [ ]:
# adjust the x axis to focus on the near centrosome area

sns.set_style("ticks")
sns.set_context("paper")
fractions_structure_1_plt = sns.relplot(x = 'distance', 
                                        y = 'percent_total_structure_1', 
                                        hue = 'rna_type', 
                                        col = 'cycle', 
                                        col_order = ['interphase', 'metaphase'], 
                                        ci = "sd", 
                                        kind="line", 
                                        data = cumulative_distribution_df);

plt.xlim(0, 1)
plt.ylim(0,100)


In [ ]:
# plot the cumulative distribution for objects containing > 4 molecules of RNA ("granules")

sns.set_style("ticks")
sns.set_context("paper")
fractions_structure_1_plt = sns.relplot(x = 'distance', 
                                        y = 'percent_granule_structure_1', 
                                        hue = 'rna_type', 
                                        col = 'cycle', 
                                        col_order = ['interphase', 'metaphase'], 
                                        ci = "sd", 
                                        kind="line", 
                                        data = cumulative_distribution_df);

plt.xlim(0)
plt.ylim(0,100)


In [ ]:
# adjust the x-axis 

sns.set_style("ticks")
sns.set_context("paper")
fractions_structure_1_plt = sns.relplot(x = 'distance', 
                                        y = 'percent_granule_structure_1', 
                                        hue = 'rna_type', 
                                        col = 'cycle', 
                                        col_order = ['interphase', 'metaphase'], 
                                        ci = "sd", 
                                        kind="line", 
                                        data = cumulative_distribution_df);

plt.xlim(0, 1)
plt.ylim(0,100)


In [ ]:

# Finally, we sometimes find that we want to capture data for a single distance point
# For example, we might want to get all the data points for what % of structure is located within 1 micron of structure 2
# To do this, we change the subset to distance == 1
# Here, we use distance == 0 to capture the % of structure 1 that overlaps w/ structure 2
# Since our dataframe contains cumulative distribution data, this point represents the % RNA / % RNA in granules
# within 1 micron

# this line creates a new dataframe containing only datapoints where the distance is equal to 0
zero_micron_df = cumulative_distribution_df.loc[cumulative_distribution_df['distance'] == 0] 

zero_micron_df['rna cycle'] = zero_micron_df['rna_type'] + ' ' + zero_micron_df['cycle']

# verify that the distance column is all 0.0
zero_micron_df

In [ ]:
# This cell will plot the percent of RNA at 0 micron as a dot plot with an overlaying box plot 
# In this example we plot the interphase data only
# Note that you can choose to save any of the plots using the save_plot function that you defined earlier

ax = sns.swarmplot(x = 'rna cycle', 
                   y = 'percent_total_structure_1', 
                   hue = 'rna_type', 
                   dodge = False,
                   order = ['cen interphase', 'gapdh interphase', 'cen metaphase', 'gapdh metaphase'],
                   data = zero_micron_df)

ax = sns.boxplot(x = 'rna cycle', 
                 y = 'percent_total_structure_1', 
                 color = "0.25", 
                 showbox = True, 
                 fliersize = 0, 
                 order = ['cen interphase', 'gapdh interphase', 'cen metaphase', 'gapdh metaphase'],
                 dodge = False, 
                 data = zero_micron_df)

# make the boxplot clear and the boundaries black
for i,box in enumerate(ax.artists):
    box.set_edgecolor('black')
    box.set_facecolor('white')

    # iterate over whiskers and median lines
    for j in range(6*i,6*(i+1)):
         ax.lines[j].set_color('black')
    
# set the y axis from 0 to 100
plt.ylim(0,100)

# make the x and y axes visible w/o the top and right frames
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

# remove the legend frame
plt.legend(frameon=False)

zero_micron_plt = ax.get_figure()


In [ ]:
subset_one_micron_total_RNA_fn = 'percent_total_RNA_at_0_micron.pdf'

save_plot(subset_one_micron_total_RNA_fn, plots_output_dir, zero_micron_plt)


## Save database tables as .csv files

This section gives you the option to save your database tables as .csv files, which allows you to view raw object data using a text editor rather than needing to have a specialized program like postgres.

By default, the data will be saved in the output/db_backups/db_csvs folder. You can change this behavior in the parameters cell


In [ ]:
# package import

import psycopg2
from psycopg2 import sql 
import os
from datetime import date


In [ ]:
# parameters 

db_csv_output_dir =  os.path.join(FILE_PATH, 'output', 'db_backups', 'db_csvs')


In [ ]:
# make folders to contain csv files if they do not already exist 

if not os.path.isdir(db_csv_output_dir):
    os.makedirs(db_csv_output_dir)


In [ ]:
# this cell gets the names of the tables you created from the postgres database

select_tables_sql = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"

conn = psycopg2.connect('postgresql://'+os.environ['POSTGRES_USER']+':'+os.environ['POSTGRES_PASSWORD']+'@'+"db"+':'+'5432'+'/'+database_name)
cur = conn.cursor()
cur.execute(select_tables_sql)
table_name_raw = cur.fetchall()
cur.close()
conn.close()

table_names = [table_name[0] for table_name in table_name_raw]

print('These are the names of your tables to be saved as .csv files:\n' + str(table_names))

In [ ]:
# this cell copies data from those tables into csv files
# it will overwrite your previously saved files

for table_name in table_names:
    table_fn = date.today().strftime('%Y.%m.%d-') + database_name + '-' + table_name + '.csv'
    table_path = os.path.join(db_csv_output_dir, table_fn)

    copy_sql_query = sql.SQL("COPY {table_name} TO STDOUT WITH CSV HEADER").format(table_name = sql.Identifier(table_name))

    conn = psycopg2.connect('postgresql://'+os.environ['POSTGRES_USER']+':'+os.environ['POSTGRES_PASSWORD']+'@'+"db"+':'+'5432'+'/'+database_name)

    cur = conn.cursor()


    with open(table_path, 'w') as f_output:
        cur.copy_expert(copy_sql_query, f_output)